# Quickstart Guide: Loading Data into Snowflake
## Introduction
This quickstart notebook demonstrates how to set up a Snowflake environment, create a database and schema, and load data from an S3 bucket into a Snowflake table using Snowflake's Snowpark Python API.

### Steps Covered:
1. Establish a Snowflake session.
2. Set up the database and schema.
3. Create a table with the specified schema.
4. Load data from an S3 bucket using a stage and file format.
5. Validate the data load by counting rows in the table.


### Step 1: Establish a Snowflake Session
In this step, we establish a session with Snowflake using the Snowpark Python API. Ensure you have the necessary credentials configured.

In [ ]:
# Import required modules and establish a session
from snowflake.snowpark.context import get_active_session

# Get active session
session = get_active_session()
session.sql_simplifier_enabled = True  # Enable SQL simplifier for optimized query execution

# Verify connection by checking the current user and Snowflake version
print("Connected to Snowflake:", session.sql('SELECT current_user(), current_version()').collect())

### Step 2: Set Up Database and Schema
We create and use the specified database and schema to ensure our environment is ready for loading data.

In [ ]:
# Database and schema configuration
database_name = 'CR_QUICKSTART'
schema_name = 'PUBLIC'

# Create and switch to the database and schema
session.sql(f'CREATE DATABASE IF NOT EXISTS {database_name}').collect()
session.sql(f'USE DATABASE {database_name}').collect()
session.sql(f'CREATE SCHEMA IF NOT EXISTS {schema_name}').collect()
session.sql(f'USE SCHEMA {schema_name}').collect()

# Print success message
print(f"Database {database_name} and Schema {schema_name} are set up.")

### Step 3: Create Table
Define and create the table structure to hold the data. We use the schema provided in the quickstart.

In [ ]:
# Define table name and create the table
table_name = 'VEHICLE'

# SQL to create the table with the specified schema
create_table_sql = f'''
CREATE OR REPLACE TABLE {database_name}.{schema_name}.{table_name} (
    C1 NUMBER(38,0),
    C2 TIMESTAMP_NTZ(9),
    C3 NUMBER(38,0),
    C4 NUMBER(38,0),
    C5 NUMBER(38,2),
    C6 NUMBER(38,0),
    C7 NUMBER(38,0),
    C8 NUMBER(38,0),
    C9 NUMBER(38,0)
);
'''

# Execute the SQL command to create the table
session.sql(create_table_sql).collect()

# Print success message
print(f"Table {table_name} created successfully.")

### Step 4: Load Data from S3 Bucket
We create a file format and stage for the S3 bucket and use the `COPY INTO` command to load the data into the table.

In [ ]:
# Define file format and stage for the S3 bucket
file_format_name = 'CSV_FORMAT'
stage_name = 'VEHICLE_STAGE'

# Replace s3 link with your own data.
s3_bucket_url = 's3://sfquickstarts/misc/demos/fraud_transactions.csv'

# Create file format for CSV
create_file_format_sql = f'''
CREATE OR REPLACE FILE FORMAT {file_format_name}
TYPE = 'CSV'
FIELD_OPTIONALLY_ENCLOSED_BY = '"'
SKIP_HEADER = 1
NULL_IF = ('NULL', '');
'''

# Create stage for S3 bucket
create_stage_sql = f'''
CREATE OR REPLACE STAGE {stage_name}
URL = '{s3_bucket_url}'
FILE_FORMAT = (FORMAT_NAME = {file_format_name});
'''

# Execute file format and stage creation
session.sql(create_file_format_sql).collect()
session.sql(create_stage_sql).collect()

# Copy data from S3 to the vehicle table
copy_into_sql = f'''
COPY INTO {database_name}.{schema_name}.{table_name}
FROM @{stage_name}
FILE_FORMAT = (FORMAT_NAME = '{file_format_name}');
'''

# Run the copy command
session.sql(copy_into_sql).collect()
print(f"Data loaded into {table_name} from {s3_bucket_url} successfully.")

### Step 5: Validate Data Load
Count the number of rows in the table to verify that the data has been loaded successfully.

In [ ]:
# Count the number of rows in the table
row_count_sql = f'''
SELECT COUNT(*) AS row_count
FROM {database_name}.{schema_name}.{table_name};
'''

# Execute the query and print the row count
row_count = session.sql(row_count_sql).collect()[0][0]
print(f"Number of rows in {table_name}: {row_count}")

In [ ]:
# Clean up by closing the session
session.close()

## Conclusion
In this quickstart notebook, we demonstrated how to:
- Set up a Snowflake session.
- Configure and create a database and schema.
- Create a table and load data from an S3 bucket.
- Validate the data load by counting rows.

You can now adapt this template to your specific datasets and schemas as needed.